<a href="https://colab.research.google.com/github/rickielin/Cardiac_Segmentation/blob/main/Cardiac_4_Cine_byChannels_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

将每份Cine的30个slices压到一个nii.gz的30个channels里。最后改成nnUNet-v1 format。命名为：**Task302** <br>

<br> Task302被存储于205 server并在该server上训练。
<br> /home1/interns/ruikai/nnUNet_v1_v2/nnUNet_raw/_nnUNet_v1_raw_data_base/nnUNet_raw_data/Task302_Cine_byChannels_only/

单张cine是不可能检测到scar 和 mvo 的 只有多张cine的相对运动才能检测到！<br>
Multiple channel 输入, single channel 输出就可以了 <br>(比如说输入三个channel rgb 图片 输出一个channel mask 图片 就可以跟ground truth 做一个loss 了)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pydicom SimpleITK

import numpy as np
import pandas as pd
from numpy import *
import seaborn as sns
import scipy.io as scio
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
import pydicom
import random
from tqdm import tqdm
import SimpleITK as sitk
import cv2
import nibabel as nib
%matplotlib inline

DATASET_FOLDER = "/content/drive/MyDrive/Temporary/Cine_LGE_samples"
# 5 example cases only

patient_case_num = []
for i in os.listdir(DATASET_FOLDER):
    patient_case_num.append(i)

patient_case_num.remove('Matching_frame_number_in_Cine_images.xlsx')
patient_case_num

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['4', '49', '26', '83', '310']

In [8]:
import glob
files = glob.glob('/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_1/Cine_new' + '/*')

files = sorted(files)  # 对文件名进行排序
files[:4]

['/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_1/Cine_new/IMG-0001-00121.dcm',
 '/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_1/Cine_new/IMG-0001-00122.dcm',
 '/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_1/Cine_new/IMG-0001-00123.dcm',
 '/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_1/Cine_new/IMG-0001-00124.dcm']

In [12]:
def load_slices_from_folder(folder_path):
    # 获取文件夹内的所有文件
    slice_files = [os.path.join(folder_path, s) for s in os.listdir(folder_path) if s.endswith('.dcm')]

    slice_files = sorted(slice_files)

    # 读取所有 DICOM 文件
    slices = [pydicom.dcmread(s) for s in slice_files]

    # 按照位置排序 slices
    # slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))

    # 获取每一张 slice 的像素数据并堆叠它们
    stacked_slices = np.stack([s.pixel_array for s in slices], axis=2)

    return stacked_slices

def save_as_nifti(data, output_path):
    # 创建 NIfTI 图像
    affine = np.eye(4)  # 使用单位矩阵作为 affine (这是一个简化，实际应用中可能需要根据 DICOM 元数据调整 affine)
    img = nib.Nifti1Image(data, affine)

    # 保存 NIfTI 图像
    nib.save(img, output_path)

if __name__ == '__main__':
    folder_path = '/content/drive/MyDrive/Temporary/Cine_LGE_samples/26/26_1/Cine_new'
    output_path = '/content/26_1.nii.gz'   # 修改为您想保存 NIfTI 文件的路径

    slices_data = load_slices_from_folder(folder_path)
    save_as_nifti(slices_data, output_path)

In [13]:
!mkdir /content/cine2lge_byChannel_data

In [14]:
# 大循环

input_path = "/content/drive/MyDrive/Temporary/Cine_LGE_samples"
dataset_path = "/content/cine2lge_byChannel_data" # 输出至

for case_num in patient_case_num:
    # 获取 _n 的个数
    Infarct_mask = scio.loadmat(DATASET_FOLDER+f"/{case_num}/Masks/InfarctMask.mat")
    Infarct_array = Infarct_mask['InfarctMask']
    mri_count = Infarct_array.shape[2]

    for _n in range(mri_count):
        folder_path = f'{input_path}/{case_num}/{case_num}_{_n+1}/Cine_new'
        output_path = f'{dataset_path}/{case_num}_{_n+1}.nii.gz'
        print(f"processing: {case_num}_{_n+1}.nii.gz")

        slices_data = load_slices_from_folder(folder_path)
        save_as_nifti(slices_data, output_path)

processing: 4_1.nii.gz
processing: 4_2.nii.gz
processing: 4_3.nii.gz
processing: 49_1.nii.gz
processing: 49_2.nii.gz
processing: 49_3.nii.gz
processing: 49_4.nii.gz
processing: 49_5.nii.gz
processing: 26_1.nii.gz
processing: 26_2.nii.gz
processing: 83_1.nii.gz
processing: 83_2.nii.gz
processing: 83_3.nii.gz
processing: 83_4.nii.gz
processing: 83_5.nii.gz
processing: 310_1.nii.gz
processing: 310_2.nii.gz
